# Azure OpenAI + ロジックアプリ (Logic Apps) で天気を取得する (実行編)

<p>始める前にカーネルの再起動を実施します</p>

__画面上部__ にある`再起動`👆を押下</br>👇の「再起動」は __スクリーンショット__ です


![カーネルの再起動](./docs/images/karnel-reboot.png)

## アシスタントと ロジック アプリの実行
### この例では、天気のデータを使って今日の天気をAzure Open AI に回答させます。
#### シーケンス図は以下のとおり


```mermaid
sequenceDiagram
    autonumber
    actor me
    participant VSCode (Notebook)
    participant Azure OpenAI Service
    participant Logic Apps
    participant MSN Weather

    me ->> +VSCode (Notebook):  今日の東京都港区港南の天気を教えてください
    VSCode (Notebook)->> +Azure OpenAI Service: 今日の東京都港区港南の天気を教えてください
    loop HealthCheck
        VSCode (Notebook)->>+VSCode (Notebook): Azure OpenAI Service 確認中
    
        Note left of Azure OpenAI Service: requires_action 
        Azure OpenAI Service->> -VSCode (Notebook): 東京都港区港南
        VSCode (Notebook)->>+Logic Apps: 東京都港区港南
        Logic Apps->>+MSN Weather: 東京都港区港南
        MSN Weather->>-Logic Apps: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
        Logic Apps->>-VSCode (Notebook): {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ... 
        VSCode (Notebook)->>+Azure OpenAI Service: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
        Azure OpenAI Service->>-VSCode (Notebook): 今日の東京都港区港南の天気は晴れです。 
        VSCode (Notebook)->>-VSCode (Notebook): Azure OpenAI Service 確認完了
    end
    VSCode (Notebook)->>+Azure OpenAI Service: 対象スレッドから応答メッセージの取得要求
    Azure OpenAI Service->>-VSCode (Notebook)　: 対象スレッドから応答メッセージの応答
    VSCode (Notebook)->>-me: 今日の東京都港区港南の天気は晴れです。
```

#### このパートは Pythonで必要なライブラリをインポートし、Azure OpenAI のインスタンスを作成

In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
from typing import Optional

load_dotenv(verbose=True)

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

#### スレッドの完了待ち
##### この関数はAzure OpenAI からの回答が到着し、Assitant API のスレッドの状態を確認する。スレッドのステータスが `requires_action` となった場合、しかるべきFunctionを実行する</p>
下図水色の部分の実装

```mermaid
sequenceDiagram
    autonumber
    actor me
    participant VSCode (Notebook)
    participant Azure OpenAI Service
    participant Logic Apps
    participant MSN Weather

    me ->> +VSCode (Notebook):  今日の東京都港区港南の天気を教えてください
    VSCode (Notebook)->> +Azure OpenAI Service: 今日の東京都港区港南の天気を教えてください
    loop HealthCheck
        rect rgb(191, 223, 255)
            VSCode (Notebook)->>+VSCode (Notebook): Azure OpenAI Service 確認中
            Note left of Azure OpenAI Service: requires_action 
            Azure OpenAI Service->> -VSCode (Notebook): 東京都港区港南
            VSCode (Notebook)->>+Logic Apps: 東京都港区港南
            Logic Apps->>+MSN Weather: 東京都港区港南
            MSN Weather->>-Logic Apps: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
            Logic Apps->>-VSCode (Notebook): {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ... 
            VSCode (Notebook)->>+Azure OpenAI Service: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
            Azure OpenAI Service->>-VSCode (Notebook): 今日の東京都港区港南の天気は晴れです。 
            VSCode (Notebook)->>-VSCode (Notebook): Azure OpenAI Service 確認完了
        end
    end
    VSCode (Notebook)->>+Azure OpenAI Service: 対象スレッドから応答メッセージの取得要求
    Azure OpenAI Service->>-VSCode (Notebook)　: 対象スレッドから応答メッセージの応答
    VSCode (Notebook)->>-me: 今日の東京都港区港南の天気は晴れです。
```

In [ ]:
def poll_run_till_completion(
    client: AzureOpenAI,
    thread_id: str,
    run_id: str,
    available_functions: dict,
    verbose: bool,
    max_steps: int = 10,
    wait: int = 3,
) -> None:
    import time
    """
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls

     client: Azure OpenAI client
     thread_id: Thread ID
     run_id: Run ID
     assistant_id: Assistant ID
     verbose: Print verbose output
     max_steps: Maximum number of steps to poll
     wait: Wait time in seconds between polls

    """

    if (client is None and thread_id is None) or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if verbose:
                print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            if run.status == "requires_action":
                tool_responses = []
                if (
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response})

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

    except Exception as e:
        print(e)

#### 応答メッセージの解析
##### Azure OpenAI からへのスレッドから実施したスレッド ID をパラメータとし、要求に対する回答をスレッドから手に入れる
下図水色の部分の実装

```mermaid
sequenceDiagram
    autonumber
    actor me
    participant VSCode (Notebook)
    participant Azure OpenAI Service
    participant Logic Apps
    participant MSN Weather

    me ->> +VSCode (Notebook):  今日の東京都港区港南の天気を教えてください
    VSCode (Notebook)->> +Azure OpenAI Service: 今日の東京都港区港南の天気を教えてください
    loop HealthCheck
        
        VSCode (Notebook)->>+VSCode (Notebook): Azure OpenAI Service 確認中
        Note left of Azure OpenAI Service: requires_action 
        Azure OpenAI Service->> -VSCode (Notebook): 東京都港区港南
        VSCode (Notebook)->>+Logic Apps: 東京都港区港南
        Logic Apps->>+MSN Weather: 東京都港区港南
        MSN Weather->>-Logic Apps: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
        Logic Apps->>-VSCode (Notebook): {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ... 
        VSCode (Notebook)->>+Azure OpenAI Service: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
        Azure OpenAI Service->>-VSCode (Notebook): 今日の東京都港区港南の天気は晴れです。 
        VSCode (Notebook)->>-VSCode (Notebook): Azure OpenAI Service 確認完了
    end
    rect rgb(191, 223, 255)
        VSCode (Notebook)->>+Azure OpenAI Service: 対象スレッドから応答メッセージの取得要求
        Azure OpenAI Service->>-VSCode (Notebook)　: 対象スレッドから応答メッセージの応答
        VSCode (Notebook)->>-me: 今日の東京都港区港南の天気は晴れです。
    end
```

In [ ]:
def retrieve_and_print_messages(
    client: AzureOpenAI, thread_id: str, verbose: bool, out_dir: Optional[str] = None
) -> any:
    """
    Retrieve a list of messages in a thread and print it out with the query and response

     client: OpenAI client
     thread_id: Thread ID
     verbose: Print verbose output
     out_dir: Output directory to save images
    @return: Messages object

    """
    from pathlib import Path

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {"user": "User query", "assistant": "Assistant response"}

        prev_role = None

        if verbose:
            print("\n\nCONVERSATION:")
        for md in reversed(messages.data):
            if prev_role == "assistant" and md.role == "user" and verbose:
                print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == "text":
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == "image_file":
                    image_data = client.files.content(mc.image_file.file_id)
                    if out_dir is not None:
                        out_dir_path = Path(out_dir)
                        if out_dir_path.exists():
                            image_path = out_dir_path / (mc.image_file.file_id + ".png")
                            with image_path.open("wb") as f:
                                f.write(image_data.read())

                if verbose:
                    if prev_role == md.role:
                        print(txt_val)
                    else:
                        print("{}:\n{}".format(display_role[md.role], txt_val))
            prev_role = md.role
        return messages
    except Exception as e:
        print(e)
        return None

#### ロジック アプリへ天気情報を要求する
##### ロジック アプリ を経由しMSN WeatherのAPIから天気情報を得る
下図水色の部分の実装

```mermaid
sequenceDiagram
    autonumber
    actor me
    participant VSCode (Notebook)
    participant Azure OpenAI Service
    participant Logic Apps
    participant MSN Weather

    me ->> +VSCode (Notebook):  今日の東京都港区港南の天気を教えてください
    VSCode (Notebook)->> +Azure OpenAI Service: 今日の東京都港区港南の天気を教えてください
    loop HealthCheck
        
        VSCode (Notebook)->>+VSCode (Notebook): Azure OpenAI Service 確認中
        Note left of Azure OpenAI Service: requires_action 
        Azure OpenAI Service->> -VSCode (Notebook): 東京都港区港南
        rect rgb(191, 223, 255)
            VSCode (Notebook)->>+Logic Apps: 東京都港区港南
            Logic Apps->>+MSN Weather: 東京都港区港南
            MSN Weather->>-Logic Apps: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
            Logic Apps->>-VSCode (Notebook): {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ... 
        end
        VSCode (Notebook)->>+Azure OpenAI Service: {"responses": {"daily": {"day":      {"cap":"Mostlysunny",...{        "cap": "Mostly sunny",   ...
        Azure OpenAI Service->>-VSCode (Notebook): 今日の東京都港区港南の天気は晴れです。 
        VSCode (Notebook)->>-VSCode (Notebook): Azure OpenAI Service 確認完了
    end
    
    VSCode (Notebook)->>+Azure OpenAI Service: 対象スレッドから応答メッセージの取得要求
    Azure OpenAI Service->>-VSCode (Notebook)　: 対象スレッドから応答メッセージの応答
    VSCode (Notebook)->>-me: 今日の東京都港区港南の天気は晴れです。
    
```

In [ ]:
def get_weather(location):
    import urllib.request
    try:
        url = os.getenv("GET_WEATHER_URL")
    
        data = {
            'location': location
        }
    
        headers = {
            'Content-Type': 'application/json'
        }
        
        req = urllib.request.Request(url, json.dumps(data).encode(), headers)
        body = None
        with urllib.request.urlopen(req) as res:
            body = json.load(res)
        
        print(type(body))
        weather_condition = body['responses']['daily']['day']['cap']
        temperature_hi = body['responses']['daily']['tempHi']
        temperature_lo = body['responses']['daily']['tempLo']
        return f"""Here is some information about the weather in {location}:
            - The weather is: {weather_condition}.
            - The highest temperature is: {temperature_hi} degrees Celsius.
            - The Lowest Temperatures is: {temperature_lo} degrees Celsius.
        """
    except Exception as e:
        print(e)

#### 天気取得のテストコード
##### `東京都港区港南`の箇所を変更し好きな場所の天気を手に入れてみましょう


In [ ]:
get_weather(location="東京都港区港南")

#### アシスタントの作成
##### Assistant のインスタンスを作成
<p>作成の中で、Assitant のが使うtools の内容を宣言する。</p>
このロジックでは `code_interpreter`と`function`の`get_weather`を使うように定義している

In [ ]:
assistant = client.beta.assistants.create(
    name="Weather Forcast",
    instructions=f"You are a helpful AI assistant who makes forecasting the weather based on data."
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. To forecast the weather, use get_weather defined in the tool."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[
        {
            "type": "code_interpreter"
        },
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "場所の検索クエリ。 都市、地域、都道府県、国、ランドマーク、郵便番号、緯度、経度を入力できます"
                        }
                    },
                    "required": [
                        "location"
                    ]
                }
            }
        }
    ],
    model=f"{os.getenv("AZURE_OPENAI_MODEL_NAME")}" #You must replace this value with the deployment name for your model.
)

<p>アシスタントインスタンスの出力</p>
とりあえず、アシスタントインスタンスを出力していみる

In [ ]:
print(assistant)

#### アシスタントの内容の出力
アシスタントインスタンスの内容をJSONでダンプしてみる

In [ ]:
print(assistant.model_dump_json(indent=2))

#### スレッドを作成
実際にAzure OpenAI に問い合わせ、解析するための実行環境(スレッド)を作成する

In [ ]:
thread = client.beta.threads.create()
print(thread)

#### スレッドに追加する質問を作成
スレッドにどんなコンテキスト（質問）にするかを定義する

In [ ]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="現在の天気を答えてください"
)

#### スレッド メッセージを一覧表示する
作成したスレッド一覧を表示してみます

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

#### スレッドを実行する
作成したスレッドを実際に実行してみます

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="今日の東京都港区港南の天気を教えてください。" 
)
available_functions = {"get_weather": get_weather}
verbose_output = True


#### スレッドの完了待ち
走らせたスレッドの完了を待ちます

In [ ]:
poll_run_till_completion(
    client=client, thread_id=thread.id, run_id=run.id, available_functions=available_functions, verbose=verbose_output
)

#### メッセージの生成
スレッドの完了後、完了したスレッドからメッセージを取得します

In [ ]:
messages = retrieve_and_print_messages(client=client, thread_id=thread.id, verbose=verbose_output)

#### メッセージの出力
取得したメッセージを出力します

In [ ]:
print(message)

[Azure OpenAI + Logic Apps で天気を取得する(準備編)](./docs/logic-apps.markdown)

[READMEに戻る](./README.markdown)